# **Model 2**


In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

#Importing and Preprocessing the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Copy of dataset.csv')
df

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values(['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

In [ ]:
df["Traffic"]= df.TrafficConditionNearby.apply(lambda x: 1 if x =='low' else (3 if x== 'high' else 2))

df["VehicleType"] = df.VehicleType.apply(lambda x: 0.5 if x=='cycle' else(1.0 if x=='bike' else (1.75 if x=='car' else 2.5)))

# Model 2's Pricing Function

In [ ]:
base_price = 10

alpha, beta, gamma, delta, eps = 7, 0.4, 2, 10, 3
n = 0.4

df['Demand'] = alpha * (df.Occupancy/df.Capacity) + beta*df.QueueLength - gamma*df.Traffic + delta*df.IsSpecialDay +eps*df.VehicleType

df['Normal_Demand'] = (df['Demand'] - df['Demand'].min()) / (df['Demand'].max() - df['Demand'].min())

df['Price']=0.0

for code in df['SystemCodeNumber'].unique():
  current = df[df['SystemCodeNumber']==code]
  for i in range(0,len(current)):
    id=current.index[i]
    df.loc[id, 'Price']= base_price*(1+n*df.loc[id, 'Normal_Demand'])

In [ ]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["Timestamp", "Occupancy", "Capacity", "QueueLength", "Traffic", "IsSpecialDay", "VehicleType", "SystemCodeNumber","Price"]].to_csv("parking_stream.csv", index=False)

# Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.

In [ ]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location
    QueueLength: int
    Traffic: int
    IsSpecialDay: int
    VehicleType: float
    SystemCodeNumber: str #For differentiating Parking Spots
    Price: float

In [ ]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [ ]:
from time import strftime
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    #day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
    #hour = data.Timestamp.dt.strptime(fmt).dt.strftime("%H")
    price=data.Price,
    syc=data.SystemCodeNumber
)


#Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [ ]:
parking_spots = df["SystemCodeNumber"].unique().tolist()

In [ ]:
# Plot
plots = []

for spot in parking_spots:
    filtered = data_with_time.filter(pw.this.SystemCodeNumber == spot)


    def price_plotter(source):
        fig = bokeh.plotting.figure(
            x_axis_type="datetime",
            height=300,
            width=600,
            title=f"Price: {spot}"
        )
        fig.line("t", "price", source=source, line_width=2, color="navy")
        fig.scatter("t", "price", source=source, size=5, color="red")
        return fig

    viz = filtered.plot(price_plotter, sorting_col="t")
    plots.append(viz)

pn.Column(*plots).servable()

In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()